In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow==1.15
!pip install keras==2.2.5
!pip install git+https://www.github.com/keras-team/keras-contrib.git

     |████████████████████████████████| 412.3MB 33kB/s 
     |████████████████████████████████| 3.8MB 39.5MB/s 
     |████████████████████████████████| 512kB 40.4MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=e9babb8d5bf76915bae16f8a9b94c89419a6069923aada2555a5779e7fed5584
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully 

In [ ]:
import numpy as np

In [ ]:
from keras.models import Sequential, Model, Input, load_model
from keras.layers import LSTM, Dense, TimeDistributed, Activation, Bidirectional, Masking, Embedding, Dropout, Flatten, concatenate, Conv1D, MaxPool1D
from keras import backend as K
from keras_contrib.layers import CRF
from keras.utils import plot_model
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras_contrib.losses import  crf_loss
from keras_contrib.metrics import crf_accuracy

Using TensorFlow backend.


In [ ]:
def building_ner(num_hidden_node, dropout, time_step, vector_length, output_lenght):
    model = Sequential()
    model.add(Masking(mask_value=0., input_shape=(25, time_step, vector_length)))

    model.add(TimeDistributed (LSTM(units=num_hidden_node, return_sequences=False, dropout=dropout, recurrent_dropout=dropout)) )

    model.add( LSTM(units=num_hidden_node, return_sequences=True, dropout=dropout, recurrent_dropout=dropout) )

    model.add(TimeDistributed(Dense(output_lenght)))
    
    model.add(Activation('softmax'))

    plot_model(model, "demo.png", show_shapes=True)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = building_ner(num_hidden_node = 32, dropout = 0.2, time_step = 25, vector_length = 137, output_lenght = 304)
model.summary()





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_1 (Masking)          (None, 25, 25, 137)       0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 25, 32)            21760     
_________________________________________________________________
lstm_2 (LSTM)                (None, 25, 32)            8320      
_________________________________________________________________
time_distributed_2 (TimeDist (None, 25, 304)           10032     
_________________________________________________________________
activation_1 (Activation)    (None, 25, 304)           0         
Total params: 40,112
Trainable params

In [ ]:
tag = np.loadtxt('/content/drive/MyDrive/data/tag_encode_input/version_2/all_error_v3.txt')
print(tag.shape)
char_encode = np.loadtxt('/content/drive/MyDrive/data/char_encode_input/version_2/all_error_v3.txt')
print(char_encode.shape)

(196750, 304)
(196750, 25)


In [ ]:
tag = tag.reshape(int(tag.shape[0]/25), 25, tag.shape[1])
print(tag.shape)
#new
char_encode = char_encode.reshape(int(char_encode.shape[0]/25), 25, char_encode.shape[1])
print(char_encode.shape)

(7870, 25, 304)
(7870, 25, 25)


In [ ]:
# chuyển từ char encode sang char embedding
LEN_OF_VOCAB = 137
shape = char_encode.shape
char_embedd = np.zeros([shape[0],shape[1],shape[2],LEN_OF_VOCAB])
for i in range(shape[0]):
  for j in range(shape[1]):
    for k in range(shape[2]):
      char_int = char_encode[i,j,k]
      char_int = char_int.astype(np.int64)
      onehot = np.zeros(LEN_OF_VOCAB)
      onehot[char_int] = 1
      char_embedd[i,j,k,:] = onehot

print(char_embedd.shape)

(7870, 25, 25, 137)


In [ ]:
import keras

cp_callback = keras.callbacks.ModelCheckpoint(filepath='/content/drive/MyDrive/data/input_output/version_1/all_error_v5.h1', verbose=1, save_best_only=True)
es_callback = keras.callbacks.EarlyStopping(monitor='acc', patience=5)

X_train, X_test, y_train, y_test = train_test_split(char_embedd, tag, test_size=0.1)

# Train the model with the new callback
model.fit(X_train, 
          y_train,
          epochs=500,
          batch_size = 64,
          validation_data=(X_test, y_test),
)
# Pass callback to training




Train on 7083 samples, validate on 787 samples
Epoch 1/500





7083/7083 [==============================] - 47s 7ms/step - loss: 3.3125 - acc: 0.5722 - val_loss: 1.9267 - val_acc: 0.6075
Epoch 2/500
7083/7083 [==============================] - 44s 6ms/step - loss: 1.7267 - acc: 0.6169 - val_loss: 1.5438 - val_acc: 0.6319
Epoch 3/500
7083/7083 [==============================] - 44s 6ms/step - loss: 1.4625 - acc: 0.6507 - val_loss: 1.3602 - val_acc: 0.6687
Epoch 4/500
7083/7083 [==============================] - 43s 6ms/step - loss: 1.3279 - acc: 0.6726 - val_loss: 1.2904 - val_acc: 0.6857
Epoch 5/500
7083/7083 [==============================] - 43s 6ms/step - loss: 1.2575 - acc: 0.6843 - val_loss: 1.2294 - val_acc: 0.6966
Epoch 6/500
7083/7083 [==============================] - 43s 6ms/step - loss: 1.2102 - acc: 0.6935 - val_loss: 1.1897 - val_acc: 0.7119
Epoch 7/500
7083/7083 [==============================] - 44s 6ms/step - loss: 1.1738 - acc: 0.6997 - val_loss: 1.1693 - val_acc: 

In [ ]:
model.save("/content/drive/MyDrive/data/input_output/version_1/all_error_v5.h1")